# Transformations &mdash; Setting up shop
> introduction to tools, techniques and libraries used in the concepts section


## Intro

Before we dive in into the concepts of 3D animations and transformations, we need to install additional tooling into our environment.

To visualize rotations, linear transformations and other relevant transformations, *OpenGL* will be used through *PyOpenGL* library. For animations we'll use *PyGame* a game development library for Python.

The following sections details how these packages can be installed and configured, and the initial usage tips regarding these tools are given.

## Setting Up Shop with *PyOpenGL* and *PyGame*

### Installing `pygame`

`pygame` library does not come with the *conda* distribution, so it has to be manually installed.

However, When trying to do simple installation you get dependency errors such as:

```bash
$ pip install pygame
...
Hunting dependencies...
WARNING: "sdl-config" failed!
```

This can be fixed by first accessing the *Software & Updates* app, and checking the *sources* option:

![Software and Updates app](../images/software-and-updates-gtk.png)

![Checking sources](../images/software-update-src.png)

Right after that, you can run:
```bash
$ sudo apt install python-dev python3-dev
$ sudo apt build-dep python-pygame
```

And then:

```bash
$ python3 -m pip install -U pygame --user
```

which seems to be same as `pip install pygame`.

The last step is to exit *VSCode* and enter again, and the `pygame` library will be available for you to import.

### Recreating the octahedron from previous chapter
In this section we will recreate the rendering of an octahedron but using *OpenGL*.

#### Setting up our `vectors` library

In chapter 1 and 2 we developed a `vector_arithmetic.py` library with all the operations related to vectors. In this section, we renamed this library as `vectors.py` for simplicity, and added a few extra functions related to the rendering section of chapter 2.

Also, some of the names have been simplified (`dot()` instead of `dot_product()`, `cross()` instead of `cross_product()`, etc.)

The library can be found in [`vectors.py`](../02-vectors-library).

| NOTE : |
| :----- |
| While the notebook approach is very conducive to the code annotation and documentation, there are issues when running the game code inside a notebook (cell gets executing, the kernel needs to restart multiple times, etc.). Please, use [octahedron_render.py](../03-octahedron-3d/) to run the octahedron render in a standalone Python programme. For this reason, Python code in the section below has been included as markdown. |

#### Creating the game loop

Now, we will include the code that will actually display the octahedron:

```python
import pygame
from pygame.locals import *
from OpenGL.GL import *
from OpenGL.GLU import *
import matplotlib.cm
from vectors import *
from math import *
```

For simplicity, we will still use the shading mechanism from `matplotlib`. Also, the `normal(...)` function that returned a vector perpendicular to the shape face will stay the same (note: it could have been added to the `vectors.py` lib):

In [ ]:
def normal(face):
    return (cross(subtract(face[1], face[0]), subtract(face[2], face[0])))

blues = matplotlib.cm.get_cmap('Blues')
def shade(face, color_map=blues, light=(1, 2, 3)):
    return color_map(1 - dot(unit(normal(face)), unit(light)))


Now, we specify the octahedron faces as we did in the previous chapter, and establish the position of our light source at `(1, 2, 3)`.

In [3]:
octahedron_faces = [ 
      [(1, 0, 0), (0, 1, 0), (0, 0, 1)],
      [(1, 0, 0), (0, 0, -1), (0, 1, 0)],
      [(1, 0, 0), (0, 0, 1), (0, -1, 0)],
      [(1, 0, 0), (0, -1, 0), (0, 0, -1)],
      [(-1, 0, 0), (0, 0, 1), (0, 1, 0)],
      [(-1, 0, 0), (0, 1, 0), (0, 0, -1)],
      [(-1, 0, 0), (0, -1, 0), (0, 0, 1)],
      [(-1, 0, 0), (0, 0, -1), (0, -1, 0)]
]

light = (1, 2, 3)

Next step consists in bootstrapping *PyGame*, establish the window size in pixels and configure *OpenGL* as the graphics engine:

```python
# pygame.init()
# switched to pygame.display.init() as the former left the process hanged

pygame.display.init()
display = (400, 400)
window = pygame.display.set_mode(display, DOUBLEBUF|OPENGL)
```

Right after that we should set up the user perspective in *OpenGL*: 

```python
# Set our perspective when looking at the the 3D scene
# 45 is the viewing angle
# 1 is the aspect ratio
# 0.1 and 50 put limits on the z coordinates that are rendered:
#   no objects further than 50 units or closer than 0.1 will be rendered
gluPerspective(45, 1, 0.1, 50.0)

# Observe the scene from 5 units up the z axis
# (-5 means move the scene down by (0, 0, -5))
glTranslatef(0.0, 0.0, -5)

# hide polygons not visible from our perspective
glEnable(GL_CULL_FACE)

# render polygons closer to us on top of those further from us
glEnable(GL_DEPTH_TEST)

# hide polygons facing us but located behind other polygons
glCullFace(GL_BACK)
```

Finally, we implement the main code that draws our octahedron.

Although not needed for this particular example, we are writing code that draws the object over and over repeatedly.

The animation loop is:
+ loop through the vectors
+ decide how to shade them
+ draw them using OpenGL
+ update the frame on the PyGame window

```python
# initializes a clock to measure time progress for PyGame
clock = pygame.time.Clock()

while True:
    # check event and process quit if user closes window
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.display.quit()
            pygame.quit()
            quit()
    
    # instruct PyGame clock to advance
    clock.tick()

    # clear previous state
    glClear(GL_COLOR_BUFFER_BIT|GL_DEPTH_BUFFER_BIT)

    # inform OpenGL we will be drawing triangles
    glBegin(GL_TRIANGLES)
    for face in octahedron_faces:
        color = shade(face, blues, light)
        for vertex in face:
            glColor3fv((color[0], color[1], color[2]))
            
            # draw current vertex from face
            glVertex3fv(vertex)
    glEnd()

    # tell PyGame next frame is ready to be rendered
    pygame.display.flip()
```

#### Changing the Perspective

The function `glTranslatef(...)` is used to tell the position from which we want to see the 3D scene we're rendering.

```python
# Observe the scene from 5 units up the z axis
# (-5 means move the scene down by (0, 0, -5))
glTranslatef(0.0, 0.0, -5)
```

There is also `glRotatef(...)` function that lets us change the angle at which we observe the scene.

```python
# rotates the whole scene by the angle theta about an axis specified by the vector (x, y, z)
glRotatef(theta, x, y, z)
``` 

The vector `(0, 0, 1)` points along the z axis, and therefore `glRotatef(30, 0, 0, 1)` will rotate the scene by 30° about the z axis.

![Rotating about the z axis](../images/octahedron_glRotatef.png)

Similarly, `glRotatef(30, 0, 1, 1)` will rotate the scene by 30° about the (0, 1, 1) which is tilted 45° between the y and z axes.

It must be noted that `glRotatef()` does not change the shading of the octahedron, because we're only moving the *position of the camera*, but as the octahedron stays in the same position, the shading will not change.

In the [octahedron_render.py](../03-octahedron-3d/) we have included a couple of functionalities associated with this behavior:
+ when clicking the right mouse button `glRotatef(30, 0, 0, 1)` will be called
+ when clicking the left mouse button `glRotatef(30, 0, 1, 1)` will be called
+ when typing `a` the automatic animation will be enabled. This will rotate the shape about the axis `(0, 1, 1)` every 10 seconds.


### Loading and Rendering the Utah teapot

In the same way we identified the vectors outlining the vertices of an octahedron, we could do the same for any other 3D object. However, artists and engineers who work on 3D models have specialized tools for this purpose, and that use a different format.

In this section we will use this approach to work with a famous 3D model: the *Utah teapot*. This is considered like the *hello world* for 3D graphics programming.

The process starts with the teapot model saved in a `teapot.off` file. This is an example of the *Object File Format*, a plaintext format that specifies the polygons that make up the surface of a 3D object, and the 3D vectors that identify the vertices of the polygon.

```
OFF
  480 448 926   # indicates number of vertices, faces and edges of the 3D  model
  0 0 0.488037  # 3D vectors for each of the vertices as x, y, z ccordinates
  ...
  4 324 306 304 317 # Specifies the faces of the model
  ..
```

In the faces section, the first number identifies the type of polygon (3 for triangle, 4 for quadrilateral, 5 for a pentagon...). The numbers that follow tell us the indices of the vertices from the previous lines that form the corners of the polygon.

Once we have the shape definition in place, we will need functions to load the vertices and polygons from that file. These are the details:
+ `load_vertices(...)` &mdash; return a list of 440 vectors, which are the vertices for the 3D model
+ `load_polygons(...)` &mdash; return a list of 448 lists, each one containing the vectors that are the vertices for that polygon.
+ `load_triangles(...)` &mdash; breaks up polygons with four or more vertices in triangles, as our rendering method is based on triangles.

Additionally, we create a function `draw_model(...)` to skip having to type the PyGame and OpenGL boilerplate code, each time we need to draw a model:

```python
def draw_model(faces, color_map=blues, light=(1, 2, 3)):
...
```

As a result of all this supporting code, we can draw the teapot with a single line:

In [2]:
from teapot import load_triangles
from draw_model import draw_model

draw_model(load_triangles())

SystemExit: 0

| NOTE: |
| :---- |
| Note that the files within the directory have been taken directly from the book code, as my vector library was giving issues. Also, a few modifications have been included in `draw_model.py` related to initialization and termination management. |